# FAISS

创建了一个 Datasets 仓库的 GitHub issues 和评论组成的数据集。在本节，我们将使用这些信息构建一个搜索引擎，帮助我们找到关于该库的最相关的 issue 的答案

In [3]:
from datasets import load_dataset

issues_dataset = load_dataset("lewtun/github-issues", split="train")

issues_dataset

Repo card metadata block was not found. Setting CardData to empty.


Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 3019
})

In [4]:
issues_dataset = issues_dataset.filter(
    lambda x: (x["is_pull_request"] == False and len(x["comments"]) > 0)
)
issues_dataset

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 808
})

In [5]:
columns = issues_dataset.column_names
columns_to_keep = ["title", "body", "html_url", "comments"]
columns_to_remove = set(columns_to_keep).symmetric_difference(columns)
issues_dataset = issues_dataset.remove_columns(columns_to_remove)
issues_dataset

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 808
})

In [6]:
test = issues_dataset.map(lambda x:{k:v for k,v in x.items() if k in ['html_url','title']},remove_columns=issues_dataset.column_names)
test

Dataset({
    features: ['html_url', 'title'],
    num_rows: 808
})

In [7]:
issues_dataset.set_format("pandas")
df = issues_dataset[:]

In [8]:
df.head()

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/issues...,Protect master branch,"[Cool, I think we can do both :), @lhoestq now...",After accidental merge commit (91c55355b634d0d...
1,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,[Hi ! I guess the caching mechanism should hav...,## Describe the bug\r\nAfter upgrading to data...
2,https://github.com/huggingface/datasets/issues...,OSCAR unshuffled_original_ko: NonMatchingSplit...,[I tried `unshuffled_original_da` and it is al...,## Describe the bug\r\n\r\nCannot download OSC...
3,https://github.com/huggingface/datasets/issues...,load_dataset using default cache on Windows ca...,"[Hi @daqieq, thanks for reporting.\r\n\r\nUnfo...",## Describe the bug\r\nStandard process to dow...
4,https://github.com/huggingface/datasets/issues...,to_tf_dataset keeps a reference to the open da...,"[I did some investigation and, as it seems, th...",To reproduce:\r\n```python\r\nimport datasets ...


In [9]:
df['comments'][0].tolist()

['Cool, I think we can do both :)',
 '@lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen to protect the master branch only from **merge commits** (see update comment above), so no need to disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted without messing up the repo history).']

In [10]:
# 使用 explode() 将这些评论中的每一条都展开成为一行
comments_df = df.explode("comments", ignore_index=True)
comments_df.head(4)

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/issues...,Protect master branch,"Cool, I think we can do both :)",After accidental merge commit (91c55355b634d0d...
1,https://github.com/huggingface/datasets/issues...,Protect master branch,@lhoestq now the 2 are implemented.\r\n\r\nPle...,After accidental merge commit (91c55355b634d0d...
2,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,Hi ! I guess the caching mechanism should have...,## Describe the bug\r\nAfter upgrading to data...
3,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,"If it's easy enough to implement, then yes ple...",## Describe the bug\r\nAfter upgrading to data...


In [11]:
from datasets import Dataset

comments_dataset = Dataset.from_pandas(comments_df)
comments_dataset


Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 2964
})

In [12]:
# comments_length 列来存放每条评论的字数
comments_dataset = comments_dataset.map(
    lambda x: {"comment_length": len(x["comments"].split())}
)
# 过滤掉简短的评论，其中通常包括“cc @lewtun”或“谢谢！”之类与我们的搜索引擎无关的内容
comments_dataset = comments_dataset.filter(lambda x: x["comment_length"] > 15)
comments_dataset

Map:   0%|          | 0/2964 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2964 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length'],
    num_rows: 2175
})

In [13]:
# 使用 issue 标题、描述和评论构建一个新的 text 列
def concatenate_text(examples):
    return {
        "text": examples["title"]
        + " \n "
        + examples["body"]
        + " \n "
        + examples["comments"]
    }


comments_dataset = comments_dataset.map(concatenate_text)
comments_dataset

Map:   0%|          | 0/2175 [00:00<?, ? examples/s]

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length', 'text'],
    num_rows: 2175
})

# 创建文本嵌入

In [14]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [15]:
import torch

device = torch.device("cuda")
model.to(device)

# GitHub issue 中的每一条记录转化为一个单一的向量，所以我们需要以某种方式“池化（pool）”或平均每个词的嵌入向量。
# 一种流行的方法是在我们模型的输出上进行 CLS 池化 ，我们只需要收集 [CLS] token 的的最后一个隐藏状态。
# 以下函数实现了这个功能，为什么只需要CLS，是因为模型最后一层的时候CLS已经有了整个句子的含义
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0] #[:,0]   模型最后一层的，所有行的第一个token也就是[CLS]

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

# 测试一篇text
embedding = get_embeddings(comments_dataset["text"][0])
embedding.shape
embedding_dataset = comments_dataset.map(
    lambda x: {"embeddings":get_embeddings(x['text']).detach().cpu().numpy()[0]}
)
# 文本嵌入转换为 NumPy 数组——这是因为当我们尝试使用 FAISS 搜索它们时，Datasets 需要这种格式

Map:   0%|          | 0/2175 [00:00<?, ? examples/s]

# 使用 FAISS 进行高效的相似性搜索


In [16]:
# !conda uninstall faiss-cpu
# !pip uninstall faiss-cpu
# !conda install -c pytorch faiss-cpu

In [17]:
import faiss
import numpy as np
!python -c "import faiss; print(dir(faiss))"
print(faiss.__file__)  # 应该输出类似 D:\Anaconda\envs\d2lc\lib\site-packages\faiss\__init__.py

# 创建随机向量
d = 64  # 向量维度
n = 1000  # 向量数量
xb = np.random.random((n, d)).astype('float32')

# 创建 IndexFlatL2 索引
index = faiss.IndexFlatL2(d)  # 这行可能会出错
print(index.is_trained)  # 应该输出 True

# 添加向量到索引
index.add(xb)
print(index.ntotal)  # 应该输出 1000

['APPROX_TOPK_BUCKETS_B16_D2', 'APPROX_TOPK_BUCKETS_B32_D2', 'APPROX_TOPK_BUCKETS_B8_D2', 'APPROX_TOPK_BUCKETS_B8_D3', 'AdditiveCoarseQuantizer', 'AdditiveQuantizer', 'AlignedTableFloat32', 'AlignedTableFloat32_round_capacity', 'AlignedTableUint16', 'AlignedTableUint16_round_capacity', 'AlignedTableUint8', 'AlignedTableUint8_round_capacity', 'AlignedTable_to_array', 'ArrayInvertedLists', 'AutoTuneCriterion', 'BinaryInvertedListScanner', 'BitstringReader', 'BitstringWriter', 'BlockInvertedLists', 'BufferList', 'BufferedIOReader', 'BufferedIOWriter', 'ByteVector', 'ByteVectorVector', 'CMax_float_partition_fuzzy', 'CMax_uint16_partition_fuzzy', 'CMin_float_partition_fuzzy', 'CMin_uint16_partition_fuzzy', 'CenteringTransform', 'CharVector', 'Cloner', 'Clustering', 'Clustering1D', 'ClusteringIterationStats', 'ClusteringIterationStatsVector', 'ClusteringParameters', 'CodePacker', 'CodePackerFlat', 'CodeSet', 'CombinerRangeKNNfloat', 'CombinerRangeKNNint16', 'ComputeCodesAddCentroidsLUT0Memor

In [18]:
# 创建一个 FAISS index（索引）
embedding_dataset.add_faiss_index(column="embeddings")
# 使用 Dataset.get_nearest_examples() 函数进行最近邻居查找
question = "How can I load a dataset offline?"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

scores, samples = embedding_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=5
)
# 收集并排序这五个最接近结果
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.head()
samples_df.sort_values("scores", ascending=False, inplace=True)

# 遍历前几行来查看我们的查询与评论的匹配程度如何
for _, row in samples_df.iterrows():
    print(f"COMMENT: {row.comments}")
    print(f"SCORE: {row.scores}")
    print(f"TITLE: {row.title}")
    print(f"URL: {row.html_url}")
    print("=" * 50)
    print()

  0%|          | 0/3 [00:00<?, ?it/s]

COMMENT: Requiring online connection is a deal breaker in some cases unfortunately so it'd be great if offline mode is added similar to how `transformers` loads models offline fine.

@mandubian's second bullet point suggests that there's a workaround allowing you to use your offline (custom?) dataset with `datasets`. Could you please elaborate on how that should look like?
SCORE: 25.505016326904297
TITLE: Discussion using datasets in offline mode
URL: https://github.com/huggingface/datasets/issues/824

COMMENT: The local dataset builders (csv, text , json and pandas) are now part of the `datasets` package since #1726 :)
You can now use them offline
```python
datasets = load_dataset('text', data_files=data_files)
```

We'll do a new release soon
SCORE: 24.555530548095703
TITLE: Discussion using datasets in offline mode
URL: https://github.com/huggingface/datasets/issues/824

COMMENT: I opened a PR that allows to reload modules that have already been loaded once even if there's no intern